### 作業
請嘗試使用 keras 來定義一個直接預測 15 個人臉關鍵點坐標的檢測網路，以及適合這個網路的 loss function


Hint: 參考前面的電腦視覺深度學習基礎

### 範例
接下來的程式碼會示範如何定義一個簡單的 CNN model

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 使用 colab 環境的同學請執行以下程式碼
%tensorflow_version 1.x # 確保 colob 中使用的 tensorflow 是 1.x 版本而不是 tensorflow 2
import tensorflow as tf
print(tf.__version__)

import os
from google.colab import drive 
drive.mount('/content/gdrive') # 將 google drive 掛載在 colob，
%cd 'gdrive/My Drive'
os.system("mkdir cupoy_cv_part4") # 可以自己改路徑
%cd cupoy_cv_part4 # 可以自己改路徑

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `1.x # 確保 colob 中使用的 tensorflow 是 1.x 版本而不是 tensorflow 2`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
1.15.0
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive
[Errno 2] No such file or directory: 'cupoy_cv_part4 # 可以自己改路徑'
/content/gdrive/My Drive


In [0]:
# 讀取資料集以及做前處理的函數
def load_data(dirname):
    # 讀取 csv 文件
    data = pd.read_csv(dirname)
    # 過濾有缺失值的 row
    data = data.dropna()

    # 將圖片像素值讀取為 numpy array 的形態
    data['Image'] = data['Image'].apply(lambda img: np.fromstring(img, sep=' ')).values 

    # 單獨把圖像 array 抽取出來
    imgs = np.vstack(data['Image'].values)/255
    # reshape 為 96 x 96
    imgs = imgs.reshape(data.shape[0], 96, 96)
    # 轉換為 float
    imgs = imgs.astype(np.float32)
    
    # 提取坐標的部分
    points = data[data.columns[:-1]].values

    # 轉換為 float
    points = points.astype(np.float32)

    # normalize 坐標值到 [-0.5, 0.5]
    points = points/96 - 0.5
    
    return imgs, points

In [5]:
cd D43

/content/gdrive/My Drive/D43


In [6]:
# 讀取資料
imgs_train, points_train = load_data(dirname = 'training.csv')
print("圖像資料:", imgs_train.shape, "\n關鍵點資料:", points_train.shape)

圖像資料: (2140, 96, 96) 
關鍵點資料: (2140, 30)


In [0]:
imgs_train = np.expand_dims(imgs_train, axis=3)

In [8]:
imgs_train.shape

(2140, 96, 96, 1)

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, MaxPool2D, BatchNormalization, Input, Activation
from keras.optimizers import SGD, Adam
from keras.models import Model

Using TensorFlow backend.


In [10]:
# 定義人臉關鍵點檢測網路
input_shape = Input((96, 96, 1))

model = Conv2D(32, (3, 3), padding="same")(input_shape)
model = BatchNormalization(axis=-1)(model)
model = Activation("relu")(model)
model = MaxPooling2D(pool_size=(2, 2))(model)

model = Conv2D(64, (3, 3), padding="same")(input_shape)
model = BatchNormalization(axis=-1)(model)
model = Activation("relu")(model)
model = Conv2D(64, (3, 3), padding="same")(input_shape)
model = BatchNormalization(axis=-1)(model)
model = Activation("relu")(model)
model = MaxPooling2D(pool_size=(2, 2))(model)

# model = Conv2D(128, (3, 3), padding="same")(input_shape)
# model = BatchNormalization(axis=-1)(model)
# model = Activation("relu")(model)
# model = Conv2D(128, (3, 3), padding="same")(input_shape)
# model = BatchNormalization(axis=-1)(model)
# model = Activation("relu")(model)
# model = MaxPooling2D(pool_size=(2, 2))(model)

model = Flatten()(model)
model = Dense(256)(model)
model = BatchNormalization()(model)
model = Activation("relu")(model)
model = Dense(30, activation=None, use_bias=True)(model)

model = Model(inputs=input_shape, outputs=model)

# 定義神經網路的輸入, hidden layer 以及輸出

# 配置 loss funtion 和 optimizer
# model.compile(loss='', optimizer='')
model.summary()
adam = Adam(lr=0.001)
#     sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(
    loss='mean_squared_error', optimizer=adam, metrics=['accuracy'])













Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 1)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 96, 96, 64)        640       
_________________________________________________________________
batch_normalization_3 (Batch (None, 96, 96, 64)        256       
_________________________________________________________________
activation_3 (Activation)    (None, 96, 96, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 147456)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)       

In [12]:
model.fit(
        imgs_train,
        points_train,
        validation_split=0.2,
        epochs=500,
        batch_size=32,
        verbose=2)

model.save('points_model.h5')

Train on 1712 samples, validate on 428 samples
Epoch 1/500
 - 4s - loss: 0.0092 - acc: 0.4433 - val_loss: 0.0969 - val_acc: 0.0607
Epoch 2/500
 - 4s - loss: 0.0048 - acc: 0.5491 - val_loss: 0.0415 - val_acc: 0.0888
Epoch 3/500
 - 4s - loss: 0.0032 - acc: 0.5964 - val_loss: 0.0289 - val_acc: 0.1145
Epoch 4/500
 - 4s - loss: 0.0025 - acc: 0.6484 - val_loss: 0.0199 - val_acc: 0.1402
Epoch 5/500
 - 4s - loss: 0.0020 - acc: 0.6822 - val_loss: 0.0099 - val_acc: 0.2290
Epoch 6/500
 - 4s - loss: 0.0017 - acc: 0.6834 - val_loss: 0.0063 - val_acc: 0.2266
Epoch 7/500
 - 4s - loss: 0.0013 - acc: 0.7249 - val_loss: 0.0054 - val_acc: 0.2453
Epoch 8/500
 - 4s - loss: 0.0012 - acc: 0.7377 - val_loss: 0.0048 - val_acc: 0.2757
Epoch 9/500
 - 4s - loss: 0.0011 - acc: 0.7383 - val_loss: 0.0048 - val_acc: 0.1916
Epoch 10/500
 - 4s - loss: 9.9815e-04 - acc: 0.7488 - val_loss: 0.0040 - val_acc: 0.2640
Epoch 11/500
 - 4s - loss: 0.0010 - acc: 0.7558 - val_loss: 0.0036 - val_acc: 0.3949
Epoch 12/500
 - 4s - lo